In [105]:
primer_nombre = "Guillermo" #@param {type: "string"}
apellido_paterno = "Garcia" #@param {type: "string"}
apellido_materno = "Martinez" #@param {type: "string"}
fecha_nacimiento = "1997-07-10" #@param {type: "date"}
ingreso_mensual = 24990  #@param {type: "number"}
dependientes = 4  #@param {type: "integer"}



if ingreso_mensual < 0:
   print('El ingreso mensual registrado es negativo ('+ str(ingreso_mensual) +'). Utilizaremos el valor positivo para tu registro y evaluación dew crédito',)
if dependientes < 0:
  print('El número de dependientes económicos registrado es negativo  ('+ str(dependientes) +'). Utilizaremos el valor positivo para tu registro y evaluación dew crédito')

usuario = {
    "name": primer_nombre, 
    "fath_lastname": apellido_paterno, 
    "moth_lastname": apellido_materno,
    "nacimiento": fecha_nacimiento,
    "rfc": "",
    "ingreso_men": abs(ingreso_mensual),
    "dependientes": abs(dependientes),
    "aprobar" : False
      }

#para la creacion del RFC vamos a trabajr con la fecha de nacimiento como un dataframe por comodidad
fecha_nacimiento = datetime.strptime(usuario["nacimiento"], '%Y-%m-%d')
date2str = lambda x : "0"+str(x) if (x<10) else x #para agregar 0 en caso de que el mes o dia sean menores a 10.
usuario["rfc"] = (usuario["fath_lastname"][0:2]+usuario["moth_lastname"][0]+usuario["name"][0]+str(fecha_nacimiento.year)[2:]+str(date2str(fecha_nacimiento.month))+str(date2str(fecha_nacimiento.day))).upper()

#aprobaremos el credito solo si gana mas de 25k al mes, o si gana entr 15-25k y tiene <3 dependientes economicos
if 25000 <= usuario["ingreso_men"]:
  usuario["aprobar"] = True
elif (25000 > usuario["ingreso_men"] > 15000) and (usuario["dependientes"] < 3):
  usuario["aprobar"] = True


Base de Datos

Se agrega o actualiza al usuario dependiendo si se encuentra ya registrado.

In [4]:
%pip install psycopg2
#%pip install -U Flask-SQLAlchemy

     ---------------------------------------- 1.2/1.2 MB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Gramirez29\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [106]:
import psycopg2

dbcredentials = {
    "hostname": "localhost", 
    "database": "api_solicitudes_db", 
    "port": 5432,
    "username": "postgres",
    "pwd": "SLBPSgramirez29"
      }

connection = None
cur = None
try:
    connection = psycopg2.connect(
        host = dbcredentials["hostname"],
        dbname = dbcredentials["database"],
        user = dbcredentials["username"],
        password = dbcredentials["pwd"],
        port = dbcredentials["port"]
    )
    cur = connection.cursor()
    
    #checking if user already in DB
    cur.execute("""SELECT "ID" FROM usuarios_registrados 
        WHERE "RFC" = '"""+usuario["rfc"]+"""' 
        AND "PRIMER_NOMBRE" = '"""+usuario["name"]+"""'
        AND "APELLIDO_PAT" = '"""+usuario["fath_lastname"]+"""'
        AND "APELLIDO_MAT" = '"""+usuario["moth_lastname"]+"""'
        """
    )
    #si el usuario ya se encuentra registrado, se actualizara su registro
    try:
        user_id = cur.fetchall()[0][0]
        #to update the data in the DB
        update_user = """UPDATE usuarios_registrados
        SET "INGRESOS_MENSUALES" = """+str(usuario["ingreso_men"])+""", "DEPENDIENTES" = """ +str(usuario["dependientes"])+""", "APROBADO" = """+str(usuario["aprobar"])+ """
        WHERE "ID" = """+str(user_id)+"""
        """
        cur.execute(update_user)

    except:
        cur.execute('SELECT "ID" FROM usuarios_registrados ORDER BY "ID" DESC LIMIT 1')
        user_id = cur.fetchall()[0][0] + 1 #adding one so that user is not dup
        #to insert the data to the DB
        insert_user = 'INSERT INTO usuarios_registrados ("ID", "PRIMER_NOMBRE", "APELLIDO_PAT", "APELLIDO_MAT", "FECHA_NAC", "RFC", "INGRESOS_MENSUALES", "DEPENDIENTES", "APROBADO") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'
        user_list=list(usuario.values())
        user_list.insert(0,user_id)
        user_values = user_list
        cur.execute(insert_user, user_values)
    
    connection.commit()
except Exception as error:
    print(error)
finally:
    if cur:
        cur.close()
    if connection:
        connection.close()

Solicitud de Crédito API

Se retorna el ID, RFC y si el usuario fue aprobado o rechazado

In [120]:
%pip install pyngrok nest_asyncio fastapi uvicorn loguru
#Flask and FastAPI run in our local server. NGrok is used to export connect our local server to the cloud so that it can be accessed worldwide through a temporal URL.


     -------------------------------------- 58.3/58.3 KB 341.0 kB/s eta 0:00:00


You should consider upgrading via the 'c:\Users\Gramirez29\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [123]:
try:
  from fastapi import FastAPI
  import nest_asyncio
  from pyngrok import ngrok
  import uvicorn
except ImportError as eImp:
  print(f"There was an error importing the next libraries: {eImp}")

app = FastAPI()

@app.get("/")
def return_API():
    json_output = """{
      "ID":"""+user_id+""",
      "RFC":"""+usuario["rfc"]+""",
      "Solicitud":"""+str(usuario["aprobar"])+"""
    }
    """
    return {"hello":"world"}



ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

t=2022-11-02T16:35:53-0600 lvl=eror msg="failed to reconnect session" obj=csess id=7da67c8e32c1 err="read tcp 100.105.174.217:53787->3.136.132.147:443: wsarecv: An existing connection was forcibly closed by the remote host."
t=2022-11-02T16:35:53-0600 lvl=eror msg="failed to reconnect session" obj=csess id=7da67c8e32c1 err="read tcp 100.105.174.217:53791->3.136.132.147:443: wsarecv: An existing connection was forcibly closed by the remote host."
t=2022-11-02T16:35:54-0600 lvl=eror msg="failed to reconnect session" obj=csess id=7da67c8e32c1 err="Get \"https://dns.google.com/resolve?cd=true&name=tunnel.us.ngrok.com&type=A\": x509: certificate has expired or is not yet valid: "
t=2022-11-02T16:35:57-0600 lvl=eror msg="failed to reconnect session" obj=csess id=7da67c8e32c1 err="read tcp 100.105.174.217:53795->3.20.27.198:443: wsarecv: An existing connection was forcibly closed by the remote host."
t=2022-11-02T16:35:58-0600 lvl=eror msg="failed to reconnect session" obj=csess id=7da67c8e32

PyngrokNgrokError: The ngrok process errored on start: x509: certificate is valid for tunnel.us.ngrok.com, not korgn.su.lennut.com.